# Integer Programming

In the last session, you learned about the JuMP ecosystem and solved simple optimization problems.
You've probably worked with JuMP to solve even more complicated LPs in 15.081.
In this session, we'll look into other types of problems, namely, **integer and nonlinear optimization problems**.

In the first half of the session, we'll focus on (Mixed) Integer Programming, which studies optimization problems in which some or all of the variables are restricted to be integers. Integer programs model situations where we need to make discrete decisions, which are frequently encountered in Operations Research.

**REMARK:** The simplest case of IP, namely, Binary Integer Linear Programming is NP-complete. So shouldn't we just go home?


## I. IP Basics

### I.1. Ryan's Unbounded Knapsack

Every morning, Ryan goes to the coffee shop and gets as much coffee as possible to be productive during the day.
There are $N$ types of coffee he can choose from, each with different caffeine content $v_i$ and price $w_i$ (you may assume that the coffee shop has an infinite supply of each coffee type).
Apparently Ryan doesn't want to go bankrupt, so he won't spend more that $C$ dollars. 
How does he choose what coffees to buy to maximize his caffeine intake and hence his productivity?

We can model Ryan's situation as a (pure) integer optimization problem:

\begin{align*}
\max& \sum_{i=1}^N v_i x_i \\
\text{s.t.}& \sum_{i=1}^N w_i x_i \leq C \\
& x_i \in \mathbb{Z}_{\geq 0} \quad \forall i = 1,\ldots,N
\end{align*}

Variable $x_i$ expresses the number of coffees of type $i$ Ryan will buy. (Ryan's favorite coffee shop only sells one-sized coffee, so all variables are constrained to be integer.)


#### A Toy Example

In particular, let's look into the following toy problem:

\begin{align*}
    \max\:& x + y + 1.5 z \\
    \text{s.t.}\:& x + 2y + 3z \leq 5.5 \\
    & x, y, z \in \mathbb{Z}_{\geq 0}
\end{align*}

How would you solve this? 

In [1]:
using JuMP, Gurobi, LinearAlgebra

# Small toy problem from above
values = [1,1,1.5]
weights = [1,2,3]
C = 5.5

# Another small problem (from JuMP documentation)
# values = [5, 3, 2, 7, 4]
# weights = [2, 8, 4, 2, 5]
# C = 10

function solve_knapsack(values, weights, C)
    N = length(values)
    knapsackModel=Model(Gurobi.Optimizer)
    @variable(knapsackModel, x[1:N]>=0, Int)
    @constraint(knapsackModel, capacity, dot(x, weights) <= C)
    @objective(knapsackModel, Max, dot(x, values))
    print(knapsackModel)
    optimize!(knapsackModel)
    return value.(x), objective_value(knapsackModel), knapsackModel
end

x_opt, val_opt, model = solve_knapsack(values, weights, C)
println("Optimal solution = $x_opt \nOptimal value = $val_opt")

Academic license - for non-commercial use only
Max x[1] + x[2] + 1.5 x[3]
Subject to
 capacity : x[1] + 2 x[2] + 3 x[3] ≤ 5.5
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[3] ≥ 0.0
 x[1] integer
 x[2] integer
 x[3] integer
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0xee7d5a27
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 6e+00]
Found heuristic solution: objective 5.0000000
Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%

User-callback calls 32, time in user-callback 0.00 

#### Modifying the Problem
During happy hour, the coffee shop sells coffee of type z with a 50% discount. Thankfully, Ryan has already computed the optimal solution before the discount, so he hopes that he can slightly modify his existing model and resolve it, taking advantage of any knowledge he already has.
    
In the latest versions of JuMP, we can modify and delete constraints as follows:

In [2]:
println("\nModel before modification:")
print(model)
println(" --> Objective value = $(objective_value(model))")

# Now let's modify the model
z = all_variables(model)[3]
con = constraint_by_name(model,"capacity")
set_normalized_coefficient(con, z, 1.5)

println("\nModel after modification:")
print(model)
println()
optimize!(model)
println(" --> Objective value = $(objective_value(model))")


Model before modification:
Max x[1] + x[2] + 1.5 x[3]
Subject to
 capacity : x[1] + 2 x[2] + 3 x[3] ≤ 5.5
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[3] ≥ 0.0
 x[1] integer
 x[2] integer
 x[3] integer
 --> Objective value = 5.0

Model after modification:
Max x[1] + x[2] + 1.5 x[3]
Subject to
 capacity : x[1] + 2 x[2] + 1.5 x[3] ≤ 5.5
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[3] ≥ 0.0
 x[1] integer
 x[2] integer
 x[3] integer

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0xf1346206
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 6e+00]

Loaded MIP start from previous solve with objective 5

Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available proc

### Ι.2. Branch and Bound Tree

Although IP solvers are often viewed as black boxes, in what follows, we'll next try to "open" the box.

For simplicity, we first consider a pure binary optimization problem over two variables (there are only two types of coffee, and Ryan can get at most one cup of each):

\begin{align*}
\max& \quad v_x x + v_y y\\
\text{s.t.}& \quad w_x x + w_y y \leq C \\
& \quad x,y \in \{0,1\}
\end{align*}

The simple way is just to consider each possible value for $x$ and $y$ and compare the cost.

![alt text](img/tree_1.png)

In the general case, this would lead to $2^N$ possible collections of items. After Ryan has examined all of them, he just chooses the best set among the ones he can afford.


Let's visualize this approach as a search tree:

![alt text](img/tree_2.png)

It's rooted at what we call the **relaxation**: none of variables have integrality enforced. As we go down leaves of the tree, we pick a variable to **branch** on, and create two descended nodes that fix that variable to one of its possible values. If we follow the tree all the way to the bottom, we reach our enumeration from before.

As we go down the arcs of the tree we restrict our problem more and more, we must have that:

>If node ``q`` is descended from node ``p``, we must have that the optimal cost of subproblem ``q`` is no more than that for node ``p``

This leads us to a powerful tool in solving these enumeration problems: 

>If I can show you that the optimal cost for subproblem ``q`` is _less_ than the optimal cost for the original problem, the same is true for any descendent of ``q``. 


That is, we can **prune** the tree and safely discard some nodes, kind of like this:

![alt text](img/tree_3.png)

#### Back to our Toy Example

Hopefully we're now familiar with how the branch and bound tree works for IP's with binary variables.
Let's turn back to our toy example that contains nonnegative integer variables and see how the branch and bound tree would be built.

\begin{align*}
    \max\:& x + y + 1.5 z \\
    \text{s.t.}\:& x + 2y + 3z \leq 5.5 \\
    & x, y, z \in \mathbb{Z}_{\geq 0}
\end{align*}

* First, we solve the LP relaxation and get $(x^*,y^*,z^*) = (5.5,0,0)$. 
* This isn't integer feasible, so we branch on $x$, which is the only non-integer variable. We construct two subproblems:
    - Subproblem 1 is:
        \begin{align*}
            \max\:& x + y + 1.5 z \\
            \text{s.t.}\:& x + 2y + 3z \leq 5 \\
            & x \leq 5
            & x, y, z \in \mathbb{Z}_{\geq 0}
        \end{align*}
        The optimal solution to this subproblem is obtained for $(x^*,y^*,z^*) = (5,0,0)$ and is integer feasible with an optimal cost of $5.$ This is the best solution we've found so far, so we update our lower bound.
    - Subproblem 2 is: 
        \begin{align*}
            \max\:& x + y + 1.5 z \\
            \text{s.t.}\:& x + 2y + 3z \leq 5 \\
            & x \geq 6
            & x, y, z \in \mathbb{Z}_{\geq 0}
        \end{align*}
        This is infeasible. 
* We've exhausted the tree, so we have our optimal solution!

The branch-and-bound scheme can end up solving many subproblems, so for it to work well, we need to *prune* large portions of the tree. 

### Ι.3. Branch and Bound Algorithm
We'll keep track of a global _lower bound_ $LB$ for our problem. Each node ``q`` will have an upper bound $UB_q$ that it inherents from its parent. If we get to the point where we have solved all subproblems (or, ideally, pruned off a great deal of them), we know that we're optimal. To do this we'll also keep track of a list $L$ of subproblems left to solve; initially, it's just the relaxation. The procedure is:

While $L$ is not empty, pick a subproblem ``q`` out of our list $L$ and solve it. 
1. ``if`` ``q`` is infeasible, ``continue``
2. ``if`` the solution is integer feasible, update the lower bound $LB$ if the cost is higher than what we had before
3. ``if``  the relaxation value is less than our global $LB$ ``continue``
4. ``else`` pick a non-integer variable $i$ and _branch_ by adding two subproblems to $L$: 
    * One with $x_i = 0$
    * Another with $x_i = 1$

Branch-and-bound is sometimes called an _implicit enumeration_ scheme because of step 3: we avoid solving any subproblems that we can prove won't produce the optimal solution.

### I.4. Ιmplementation of the Branch and Bound Algorithm in Gurobi

The "magic" of modern MIP solvers largely comes down to pruning massive portions of the tree. Some of this is essentially beyond your control, but there are certain things which you can do. This is the topic of Part II of this IP crash course.

In what follows, we focus on **Gurobi**, a commercial solver that solves Mixed Integer LPs/QPs/QCQPs. (You can get the full picture of what solvers JuMP supports and what types of problems you can solve with each of them by visiting http://www.juliaopt.org/JuMP.jl/latest/installation/ and scrolling a bit down.)

What are the ingredients of Gurobi's branch and bound implementation?
 - **Presolve**: reduce problem size via removal of redundant constraints and variable substitutions.
 - **Sophisticated Implementations of Continuous Optimization Methods**: simplex-based, barrier-based.
 - **Cutting Planes**: over the course of the solution process, add cuts that tighten the model and remove potential undesired fractional solution. Here is an example:
     - Consider the constraint $6 x_1 + 5 x_2 + 7 x_3 + 4 x_4 + 5 x_5 \leq 15$, where $x_1$ through $x_5$ are restricted to be binary. 
     - Suppose in addition that we have just solved an LP relaxation and that these variables take the following values in this LP relaxation: $x_1 = 0, x_2 = 1, x_3 = x_4 = x_5 = \frac{3}{4}$. 
     - This undesirable solution can be excluded with the following observation: since $7 + 4 + 5 = 16 > 15$, it is not possible that $x_3 = x_4 = x_5 = 1$, and hence that the following new inequality is a valid addition to the given MIP: $x_3 + x_4 + x_5 \leq 2$. Since $\frac{3}{4} + \frac{3}{4} + \frac{3}{4} = \frac{9}{4} > 2$, the new inequality cuts off the current (fractional and therefore infeasible) solution, but importantly does not cut off any feasible integer solutions.
     - Consequently, the relaxations solved from this point are of higher quality.
 - **Heuristics**: e.g., randomized rounding.
 - **Branch Variable Selection**

### Ι.5. Understanding Gurobi's Output
First, it solves the LP relaxation and reports back:
```
Root relaxation: objective 4.014179e+00, 18 iterations, 0.00 seconds
```
Now it explores the branch-and-bound tree, and updates us as it goes along. Let's look at just the first line:
```
    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.01418    0    7    2.35937    4.01418  70.1%     -    0s
```
We see that the information is broken down into four main columns:

1. ``Nodes``: Global node information
    * how many nodes have we looked at
    * how many do we have in our queue
2. ``Current Node``
    * objective
    * depth in the tree
    * number of noninteger variables in the solution
3. ``Objective Bounds``
    * Best incumbent (lower bound)
    * node upper bound
    * the gap between the two
4. ``Work``
    * average simplex iterations per node
    * total elapsed time

Finally, we get a neat summary of the cutting planes Gurobi found useful:
```
Cutting planes:
  Gomory: 3
  Cover: 2
  MIR: 5
```
All told, we explored 190  nodes, much less than the $2^{100}$ we were worried about. All this only took 698 simplex iterations and 0.21 seconds.

Now what about those ``H``s that appear? That tells us that Gurobi ran a heuristic and found a new best solution. You can see for yourself, as the incumbent value increases while the bound remains the same:
```
    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.01418    0    7    2.35937    4.01418  70.1%     -    0s
H    0     0                       3.3780464    4.01418  18.8%     -    0s
```
You'll also sometimes see a ``*`` instead of the ``H``, which says that the feasible solution came from branching instead of heuristics.

## ΙΙ. Advanced IP

Now that we've mastered the basics, we'll look into more advanced stuff; we'll try to interact with the solver and intervene in the solving process.

### II.1. Lazy Constraints in Ryan's Unbounded Knapsack

Ryan is willing to sacrifice some caffeine intake to switch between espresso, cold brew, and flat white.
In particular, he wants the maximum difference between the selected quantities of any two types of coffee to be no more that $\mu$.
This requirement leads to $2 {n \choose 2}$ constraints of the form: $x_i - x_j \leq \mu \ \forall i\neq j$.
Instead of enumerating all of them and adding them a priori to the model, we may use a technique known as **lazy constraints**.

In particular, every time our solver reaches a new solution, for example with a heuristic or by solving a problem at a node in the branch and bound tree, it will give the user the chance to provide constraint(s) that would make the current solution infeasible.


#### Reasons to Use Lazy Constraints

- The model involves a large number of constraints, many of which will most likely be redundant or non-binding near an optimal solution. In many cases, it can even be intractable to generate all the constraints. 
- In some cases, we may be unable to identify all constraints at the time the model is specified. The feasibility and optimality cuts generated during Benders decomposition fall into this category; we discover them by solving one or more subproblems at certain points in the search for the solution to the master problem.


#### Implementing Lazy Constraints

MIP solvers implement lazy constraints via a technique known as **solver callback**.
JuMP currently supports **solver-independent callbacks** for CPLEX, GLPK, and Gurobi.

**REMARK:** Part of the major changes JuMP underwent between versions 0.18 and 0.19 was the removal of solver-independent callbacks. Support for solver-independent callbacks was restored later. 

There are three important steps to providing a lazy constraint callback in JuMP. 

- **Callback function**: a function that will analyze the current solution. This function takes as argument a reference to the callback management code inside JuMP. Currently, the only thing we may query in a callback is the primal value of the variables using the function "callback_value". If we need any other information, we may use a **solver-dependent** callback instead (for an example, look here https://discourse.julialang.org/t/solver-dependent-callbacks-in-jump-how-to-do-it-right/32130).

- **Lazy constraint**: after analyzing the current solution, we generate a new constraint using the 

        "con = @build_constraint(...)" 
    macro and submit it to the model via the MOI interface 
        
        "MOI.submit(model, MOI.LazyConstraint(cb), con)."
        
- **Lazy constraint callback**: we again use the MOI interface to tell the solver which function should be used for lazy constraint generation 

        "MOI.set(model, MOI.LazyConstraintCallback(), my_callback)."

In [4]:
function solve_fair_knapsack(values, weights, C, max_diff)
    N = length(values)
    fairKnapsackModel=Model(Gurobi.Optimizer)
    @variable(fairKnapsackModel, x[1:N]>=0, Int)
    @constraint(fairKnapsackModel, dot(x, weights) <= C)
    @objective(fairKnapsackModel, Max, dot(x, values))
    lazy_called = false
    function my_callback(cb) # what is cb? what data can we access during callback?
        lazy_called = true
        x_vals = callback_value.(Ref(cb), x)
        # First, let's find a violated constraint!
        i_max, i_min = argmax(x_vals), argmin(x_vals)
        con = @build_constraint(x[i_max] - x[i_min] <= max_diff)
        MOI.submit(fairKnapsackModel, MOI.LazyConstraint(cb), con)
    end 
    MOI.set(fairKnapsackModel, MOI.LazyConstraintCallback(), my_callback)
    print(fairKnapsackModel)
    println("\n*** Callback called? $lazy_called\n\n")
    optimize!(fairKnapsackModel)
    println("\n*** Callback called? $lazy_called\n\n")
    return value.(x), objective_value(fairKnapsackModel), fairKnapsackModel
end

max_diff = 2
xf_opt, valf_opt, model = solve_fair_knapsack(values, weights, C, max_diff)
println("Optimal solution = $xf_opt \nOptimal value = $valf_opt")


Academic license - for non-commercial use only
Max x[1] + x[2] + 1.5 x[3]
Subject to
 x[1] + 2 x[2] + 3 x[3] ≤ 5.5
 x[1] ≥ 0.0
 x[2] ≥ 0.0
 x[3] ≥ 0.0
 x[1] integer
 x[2] integer
 x[3] integer

*** Callback called? false


Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0xee7d5a27
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 6e+00]
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros
Variable types: 0 continuous, 3 integer (1 binary)
Found heuristic solution: objective 1.5000000

Root relaxation: objective 4.000000e+00, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.50000    0    1    1.50000  

### II.2. Callback Types

JuMP 0.21 supports three types of callbacks:

- **Lazy constraints**: See previous section.
        
        
- **User cuts**: User cuts provide a way for the user to tighten the LP relaxation using problem-specific knowledge that the solver cannot infer from the model and hence cannot utilize when generating cuts like the ones we saw earlier (Gurobi's cutting planes component).

      MOI.submit(model, MOI.UserCut(cb), con)
      
      MOI.set(model, MOI.UserCutCallback(), my_callback_function)

    - Importantly, user cuts **should not change the set of integer feasible solutions** and can only remove fractional solutions; if we add a cut that removes an integer solution, the solver may return an incorrect solution. **That's the main difference between user cuts and lazy constraints.**

    - Just like with lazy constraints, when a MIP solver reaches a new node in the branch-and-bound tree, it will give the user the chance to provide cuts to make the current relaxed (fractional) solution infeasible in the hopes of obtaining an integer solution.
    
    - Generally speaking, solvers can add general purpose cuts (e.g., CG, split, MIR) and structure specific cuts (e.g., knapsack cover, clique) better than we can. However, we are better at adding problem specific cuts. Therefore, when trying to improve bound quality, a good place to start is identifying problem structure which a solver hasn't found, and exploiting this problem structure.
   
    
    
- **Heuristic solutions**: By heuristic solution we refer to the method that the solver applies during the solution process to find integer solutions quicker than plain branch-and-bound would and tighten the bound, allowing us to fathom nodes quicker and to tighten the integrality gap.
    
      status = MOI.submit(model, MOI.HeuristicSolution(cb), [x], [floor(Int, x_val)]) # accept/reject/unknown
            
      MOI.set(model, MOI.HeuristicCallback(), my_callback_function)

    - Solvers' heuristics are based on neighborhood search (e.g., flipping binary variables, fix some variables and solve a smaller MILP), rounding or "polishing" existing solutions.

    - This callback enables us to add heuristics of our own if we have some special insight into the problem structure that the solver is not aware of. For instance, if we're solving a knapsack problem, one simple heuristic is to add a **greedy solution** where we iteratively add the best available item to the sack until you run out of room. This will often be a very good solution, and is a simple example of a problem-specific heuristic scheme.


- Previous versions of JuMP also supported **informational callbacks**, which were used to track solver progress without actually changing the algorithm by adding cuts or heuristic solutions.




# Credit + References

This material is adapted from previous versions of this course, which have been designed by numerous ORC students.

Some of the sources used to create this year's version include:
 - JuMP documentation
 - Gurobi documentation
 - https://orinanobworld.blogspot.com/2012/08/user-cuts-versus-lazy-constraints.html